### Pain Score evaluation Frame level (Select pose)

In [1]:
from sklearn.svm import SVC
from feature_process import *
import joblib
import numpy as np
import os

In [2]:
def select_pose(feat,labels, sel=None, window=6):
    sel=np.arange(10)
    # sel=[1,5,7,8,9]
    # sel=[2,4,6,9]
    newfeat = []
    for i in range(len(labels)*window):
        f = feat[i,:]
        if labels[i//window] in sel:
            newfeat.append(f)
    return np.array(newfeat)

In [4]:
cap_basal_feats = []
cap_treat_feats = []
sng_basal_feats = []
sng_treat_feats = []
ro_basal_feats = []
ro_treat_feats = []
root = r'C:\Users\x\Desktop\final\data/old1_landmark7/'
poseroot = r'C:\Users\x\Desktop\final\data\old1_bsoidfeat/'
files = os.listdir(root)
posefiles = os.listdir(poseroot)
for i in range(len(files)):
    file = files[i]
    dlc1 = dlc(root+file, True, 7)
    feat = count_dist(dlc1.raw) # feature from landmark 7 
    # feat = combine_feat(dlc1, sel_dist=[[0,3],[3,4],[1,2]], sel_ang=[[0,1,3],[1,3,4]], sel_coord=[], normalize=[0,1])
    posefile = posefiles[i]
    [posefeat,labels] = joblib.load(poseroot+posefile)
    labels = labels[0]
    if file.find('Cap')!=-1:
        if file.find('basal')!=-1:
            cap_basal_feats.append(select_pose(feat,labels))
        if file.find('treat')!=-1:
            cap_treat_feats.append(select_pose(feat,labels))
    elif file.find('pH5.2')!=-1:
        if file.find('basal')!=-1:
            sng_basal_feats.append(select_pose(feat,labels))
        if file.find('treat')!=-1:
            sng_treat_feats.append(select_pose(feat,labels))
    else:
        if file.find('basal')!=-1:
            ro_basal_feats.append(select_pose(feat,labels))
        if file.find('treat')!=-1:
            ro_treat_feats.append(select_pose(feat,labels))


In [6]:
featnum = 8
x_train = np.zeros([0,featnum])
y_train = []
x_test = np.zeros([0,featnum])
y_test = []
bshuffles = []
tshuffles = []
for i in range(len(cap_basal_feats)):
    #basal
    feat = cap_basal_feats[i]
    ##shuffle
    ind = np.arange(len(feat))
    np.random.shuffle(ind)
    bshuffles.append(ind)
    feat = feat[ind,:]
    sp = len(feat)//2
    x_train = np.concatenate([x_train,feat[:sp,:]])
    y_train = y_train + [-1] * len(cap_basal_feats[i][:sp,:])
    x_test = np.concatenate([x_test,cap_basal_feats[i][sp:,:]])
    y_test = y_test + [-1] * len(cap_basal_feats[i][sp:,:])
    feat = cap_treat_feats[i]
    #treat
    feat = cap_treat_feats[i]
    ##shuffle
    ind = np.arange(len(feat))
    np.random.shuffle(ind)
    tshuffles.append(ind)
    feat = feat[ind,:]
    sp = len(feat)//2
    x_train = np.concatenate([x_train,feat[:sp,:]])
    y_train = y_train + [1] * len(feat[:sp,:])
    x_test = np.concatenate([x_test,feat[sp:,:]])
    y_test = y_test + [1] * len(feat[sp:,:])
# add RO
train_split = 10 # 10%
if train_split != 0:
    for i in range(len(ro_basal_feats)):
        sp = len(ro_basal_feats[i])//train_split
        x_train = np.concatenate([x_train,ro_basal_feats[i][:sp,:]])
        y_train = y_train + [-1] * len(ro_basal_feats[i][:sp,:])
        x_test = np.concatenate([x_test,ro_basal_feats[i][sp:,:]])
        y_test = y_test + [-1] * len(ro_basal_feats[i][sp:,:])
        sp = len(ro_treat_feats[i])//train_split
        x_train = np.concatenate([x_train,ro_treat_feats[i][:sp,:]])
        y_train = y_train + [-1] * len(ro_treat_feats[i][:sp,:])
        x_test = np.concatenate([x_test,ro_treat_feats[i][sp:,:]])
        y_test = y_test + [-1] * len(ro_treat_feats[i][sp:,:])
# add Sng
train_split = 10 # 10%
if train_split != 0:
    for i in range(len(sng_basal_feats)):
        sp = len(sng_basal_feats[i])//train_split
        x_train = np.concatenate([x_train,sng_basal_feats[i][:sp,:]])
        y_train = y_train + [-1] * len(sng_basal_feats[i][:sp,:])
        x_test = np.concatenate([x_test,sng_basal_feats[i][sp:,:]])
        y_test = y_test + [-1] * len(sng_basal_feats[i][sp:,:])

In [7]:
model = SVC(kernel='rbf', C=1000)
model = model.fit(x_train,y_train)

In [8]:
# Cap train
x_train = np.zeros([0,featnum])
y_train = []
for i in range(len(cap_basal_feats)):
    feat = cap_basal_feats[i]
    feat = feat[bshuffles[i]]
    sp = len(feat)//2
    x_train = np.concatenate([x_train,feat[:sp,:]])
    y_train = y_train + [1] * len(feat[:sp,:])
print('Accuracy = ', model.score(x_train,y_train))
x_train = np.zeros([0,featnum])
y_train = []
for i in range(len(cap_treat_feats)):
    feat = cap_treat_feats[i]
    feat = feat[tshuffles[i]]
    sp = len(feat)//2
    x_train = np.concatenate([x_train,feat[:sp,:]])
    y_train = y_train + [1] * len(feat[:sp,:])
print('Accuracy = ', model.score(x_train,y_train))
# Cap test
x_test = np.zeros([0,featnum])
y_test = []
for i in range(len(cap_basal_feats)):
    feat = cap_basal_feats[i]
    feat = feat[bshuffles[i]]
    sp = len(feat)//2
    x_test = np.concatenate([x_test,feat[sp:,:]])
    y_test = y_test + [1] * len(feat[sp:,:])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,featnum])
y_test = []
for i in range(len(cap_basal_feats)):
    feat = cap_treat_feats[i]
    feat = feat[tshuffles[i]]
    sp = len(feat)//2
    x_test = np.concatenate([x_test,feat[sp:,:]])
    y_test = y_test + [1] * len(feat[sp:,:])
print('Accuracy = ', model.score(x_test,y_test))
# RO
x_test = np.zeros([0,featnum])
y_test = []
for i in range(len(ro_basal_feats)):
    x_test = np.concatenate([x_test,ro_basal_feats[i]])
    y_test = y_test + [1] * len(ro_basal_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,featnum])
y_test = []
for i in range(len(ro_basal_feats)):
    x_test = np.concatenate([x_test,ro_treat_feats[i]])
    y_test = y_test + [1] * len(ro_treat_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
# Sng
x_test = np.zeros([0,featnum])
y_test = []
for i in range(len(sng_basal_feats)):
    x_test = np.concatenate([x_test,sng_basal_feats[i]])
    y_test = y_test + [1] * len(sng_basal_feats[i])
print('Accuracy = ', model.score(x_test,y_test))
x_test = np.zeros([0,featnum])
y_test = []
for i in range(len(sng_basal_feats)):
    x_test = np.concatenate([x_test,sng_treat_feats[i]])
    y_test = y_test + [1] * len(sng_treat_feats[i])
print('Accuracy = ', model.score(x_test,y_test))

Accuracy =  0.09083888406216817
Accuracy =  0.735973597359736
Accuracy =  0.0990121311580912
Accuracy =  0.7334024100084428
Accuracy =  0.15332998661311914
Accuracy =  0.30163230240549826
Accuracy =  0.1945289454277286
Accuracy =  0.36365531619179986
